In [11]:
import numpy as np
from pathlib import Path
import pandas as pd

pd.set_option('display.max_rows', 15)

file = Path(r"C:\Users\Administrator\Desktop\pandas进阶修炼")
df = pd.read_csv(file.joinpath("东京奥运会奖牌数据.csv"))
# df.info()
# 5-1 数据修改
# 将原 df 列名 Unnamed: 2、Unnamed: 3、Unnamed: 4 修改为 金牌数、银牌数、铜牌数
df.rename({
    'Unnamed: 2': '金牌数',
    'Unnamed: 3': '银牌数',
    'Unnamed: 4': '铜牌数',
}, axis=1, inplace=True)
df

# 2 - 数据修改｜行索引
# 将第一列（排名）设置为索引
df.set_index("排名", inplace=True)
df
# 3 - 数据修改｜修改索引名
# 修改索引名为 金牌排名
df.rename_axis("金牌排名", inplace=True)
df
# 4 - 数据修改｜修改值
# 将 ROC（第一列第五行）修改为 俄奥委会
df.applymap(lambda x: '俄奥委会' if x == 'ROC' else x)

# 5 - 数据修改｜替换值（单值）
# 将金牌数列的数字 0 替换为 无
# df['金牌数'] = df['金牌数'].map(lambda x: '无' if x==0 else x)
df['金牌数'].replace(0, '无', inplace=True)
df

# 6 - 数据修改｜替换值（多值）
# 同时替换
# 将 无 替换为 缺失值
# 将 0 替换为 None
# 注意：缺失值的 Nan 该怎么生成？
df.replace(['无', 0], [np.nan, 'None'], inplace=True)
df

# 7 - 数据查看
# 查看各列数据类型
df.dtypes

# 8 - 数据修改｜修改类型
# 将 金牌数 列类型修改为 int
df.fillna(0).astype({'金牌数': int})

# 9 - 数据增加｜新增列（固定值）
# 重新加载数据 并 新增一列 比赛地点，值为 东京
df = df.assign(比赛地点='东京')
# df['比赛地点'] = '东京'
# df

# 10 - 数据增加｜新增列（计算值）
# 新增一列 金银牌总数列，值为该国家金银牌总数
df = df.replace('None',0)
df['金银牌总数']  = df['金牌数']  + df['银牌数']
df

# 11 -数据增加｜新增列（比较值）
# 新增一列 最多奖牌数量 列，值为该国 金、银、铜 牌数量中最多的一个奖牌数量

# 例如美国银牌最多，则为41，中国为38
# df[['金牌数', '银牌数', '铜牌数']].fillna(0).astype(int).idxmax(axis=1)
# df.bfill(axis=1)[["金牌数", "银牌数",'铜牌数']].max(1)
def max_str(s: pd.Series):
    num = s.max()
    name = s.idxmax()
    return f'{name}_{num}'
df[['金牌数', '银牌数', '铜牌数']].fillna(0).astype(int).apply(max_str, axis=1)

# 12 - 数据增加｜新增列（判断值）
# 新增一列 金牌大于30
# 如果一个国家的金牌数大于 30 则值为 是，反之为 否
df['金牌大于30']  = np.where(df['金牌数'] > 30, '是', '否')
df

# 13 - 数据增加｜增加多列
# 新增两列，分别是
# 金铜牌总数（金牌数+铜牌数）
# 银铜牌总数（银牌数+铜牌数）

df = df.assign(金铜牌总数=df.金牌数 + df.铜牌数,
         银铜牌总数=df.银牌数+df.铜牌数) 
df

# 14 - 数据增加｜新增列（引用变量）
# 新增一列金牌占比，为各国金牌数除以总金牌数（gold_sum）
gold_sum = df['金牌数'].sum()
df.eval(f'金牌占比 = 金牌数 / {gold_sum}', inplace=True)
df


# 15 - 数据增加｜新增行（末尾追加）
# 在 df 末尾追加一行，内容为 0,1,2,3... 一直到 df 的列长度
df.iloc[-1, :] = [i for i in range(len(df.columns))]
df

# 16 -数据增加｜新增行（指定位置）
# 在第 2 行新增一行数据，即美国和中国之间。
# 数据内容同上一题
df1 = df.iloc[:1, :]
df2 = df.iloc[1:, :]
df3 = pd.DataFrame([[i for i in range(len(df.columns))]], columns=df.columns)
df_new = pd.concat([df1, df3, df2], ignore_index=True)
# df_new

# 17 - 数据删除｜删除行
# 删除 df 第一行
df.drop(1)

# 18 - 数据删除｜删除行（条件）
df.drop(df[df.金牌数<20].index)

# 19 - 数据删除｜删除列
# 删除刚刚新增的 比赛地点 列
df.drop(columns=['比赛地点'])

# 20 - 数据删除｜删除列（按列号）
# 删除 df 的 7、8、9、10 列
df.drop(df.columns[[7,8,9,10]], axis=1)

# 5-2 数据筛选
# 以下所有答案要求返回的是一个 dataframe 而不是 Series 这样可以直接存储为 Excel 等格式的文件！

df.rename(columns={'Unnamed: 2':'金牌数',
                  'Unnamed: 3':'银牌数',
                  'Unnamed: 4':'铜牌数'},inplace=True)

# 22 - 筛选列｜通过列号
# 提取第 1、2、3、4 列
df.iloc[:, [1, 2, 3, 4]]

## 23 - 筛选列｜通过列名
# 提取 金牌数、银牌数、铜牌数 三列
df[['金牌数','银牌数','铜牌数']]

# 24 - 筛选列｜条件（列号）
# 筛选全部 奇数列
df.iloc[:,[i%2==1 for i in range(len(df.columns))]]

# 25 - >筛选列｜条件（列名）
# 提取全部列名中以 “数” 结尾的列
df.loc[:, df.columns.str.contains('数')]
# df.loc[:, df.columns.str.endswith('数')]

# 26 - 筛选列｜组合（行号+列名）
# 提取倒数后三列的10-20行
df.loc[10:20, '总分':] 

# 27 -筛选行｜通过行号
# 提取第 10 行
df.iloc[10, :]

# 28 - 筛选行｜通过行号（多行）
# 提取第 10 行之后的全部行
(df.loc[1:9, :] == df.iloc[0:9, :]).all().all()
# df.loc[9:]
df.iloc[10:, :]

# 29 - 筛选行｜固定间隔
# 提取 0-50 行，间隔为 3
df.iloc[0:50:3, :]

# 30 - 筛选行｜判断（大于）
# 提取 金牌数 大于 30 的行
df.loc[df['金牌数']>30]

# 31 - >筛选行｜判断（等于）
# 提取 金牌数 等于 10 的行
df.loc[df['金牌数'] == 10]

# 32 - 筛选行｜判断（不等于）
# 提取 金牌数 不等于 10 的行
df.loc[~(df['金牌数'] == 10)]

# 33 - 筛选行｜条件（指定行号）
# 提取全部 奇数行
df[[i%2==1 for i in range(len(df.index))]]

# 34 - 筛选行｜条件（指定值）
# 提取 中国、美国、英国、日本、巴西 五行数据
# df.query("国家奥委会 in ['中国', '美国', '英国', '日本', '巴西']")
df.loc[df['国家奥委会'].isin(['中国','美国','英国','日本','巴西'])]

# 35 - 筛选行｜多条件
# 在上一题的条件下，新增一个条件：金牌数小于30
df.query("国家奥委会 in ['中国', '美国', '英国', '日本', '巴西'] & 金牌数 < 30")
# df.loc[(df['金牌数'] < 30) & (df['国家奥委会'].isin(['中国','美国','英国','日本','巴西']))]

# 36 -筛选行｜条件（包含指定值）
# 提取 国家奥委会 列中，所有包含 国的行
df[df['国家奥委会'].str.contains('国').fillna(False)]

# 37 - 筛选某行某列
# 提取 第 0 行第 2 列
df.iloc[0:1,[1]]

# 38 - 筛选多行多列
# 提取 第 0-2 行第 0-2 列
df.iloc[0:2,[0,1]]

# 39 - 筛选值｜组合（行号+列号）
# 提取第 4 行，第 4 列的值
df.at[4,'金牌数']

# 41 - 筛选值｜条件
# 提取 国家奥委会 为 中国 的金牌数
df.loc[df['国家奥委会'] == '中国']['金牌数']

# 42 - 筛选值｜query
# 使用 query 提取 金牌数 + 银牌数 大于 15 的国家
df.query('金牌数+银牌数 > 15')

# 43 - 筛选值｜query（引用变量）
# 使用 query 提取 金牌数 大于 金牌均值的国家
gold_mean = df['金牌数'].mean()
df.query(f'金牌数 > {gold_mean}').style.format({'金牌占比': '{:.2f}%'})

,国家奥委会,金牌数,银牌数,铜牌数,总分,按总数排名,国家奥委会代码,比赛地点,金银牌总数,金牌大于30,金铜牌总数,银铜牌总数,金牌占比
金牌排名,,,,,,,,,,,,,
1,美国,39.000000,41,33,113,1,USA,东京,80.000000,是,72.000000,74,0.11%
2,中国,38.000000,32,18,88,2,CHN,东京,70.000000,是,56.000000,50,0.11%
3,日本,27.000000,14,17,58,5,JPN,东京,41.000000,否,44.000000,31,0.08%
4,英国,22.000000,21,22,65,4,GBR,东京,43.000000,否,44.000000,43,0.06%
5,ROC,20.000000,28,23,71,3,ROC,东京,48.000000,否,43.000000,51,0.06%
6,澳大利亚,17.000000,7,22,46,6,AUS,东京,24.000000,否,39.000000,29,0.05%
7,荷兰,10.000000,12,14,36,9,NED,东京,22.000000,否,24.000000,26,0.03%
8,法国,10.000000,12,11,33,10,FRA,东京,22.000000,否,21.000000,23,0.03%
9,德国,10.000000,11,16,37,8,GER,东京,21.000000,否,26.000000,27,0.03%
